# `bitquery` code example

Related Resources:
- <https://bitquery.io/blog/graphql-with-python-javascript-and-ruby#Getting_API_Key>
- <https://bitquery.io/blog/eth2-api#Daily_Blocks_and_Proposers_count>

In [15]:
import datetime

import pandas as pd
import plotly.graph_objects as go
import requests
from plotly.subplots import make_subplots

In [2]:
def run_query(query): 
    headers = {'X-API-KEY': 'BQYy1WwTGPUPPeWYZeWDaQx5gghLlgEi'}
    request = requests.post('https://graphql.bitquery.io/',
                            json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}. {}'.format(request.status_code,query))

In [3]:
query = """
{
  ethereum2 {
    deposits(options: {asc: "date.date"}) {
      date: date {
        date(format: "%Y-%m-%d")
      }
      count: count
      amount
    }
  }
}
"""

In [4]:
result = run_query(query=query)

In [5]:
deposit_history = result.get('data').get('ethereum2').get('deposits')

In [11]:
deposit_history_flatten = []

for item in deposit_history:
    date = item.get('date').get('date')
    count = item.get('count')
    amount = item.get('amount')

    deposit_history_flatten.append(
        dict(
            date=date,
            count=count,
            amount=amount
        )
    )

deposit_history_df = pd.DataFrame(deposit_history_flatten)
deposit_history_df['date'] = deposit_history_df['date'].astype('datetime64[D]')

# aggregate
deposit_history_df['cumsum_amount'] = deposit_history_df['amount'].cumsum()

print(deposit_history_df.info())
display(deposit_history_df)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775 entries, 0 to 774
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           775 non-null    datetime64[ns]
 1   count          775 non-null    int64         
 2   amount         775 non-null    float64       
 3   cumsum_amount  775 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 24.3 KB
None


,date,count,amount,cumsum_amount
0,2020-12-01,6451,206401.0,206401.0
1,2020-12-02,1644,52577.0,258978.0
2,2020-12-03,1557,49824.0,308802.0
3,2020-12-04,547,17504.0,326306.0
4,2020-12-05,2574,82276.0,408582.0
...,...,...,...,...
770,2023-01-10,336,10128.0,15282983.0
771,2023-01-11,444,13440.0,15296423.0
772,2023-01-12,1025,31552.0,15327975.0
773,2023-01-13,225,6464.0,15334439.0


In [18]:
# some primary visualization
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

fig.add_trace(
    go.Line(
        x=deposit_history_df['date'],
        y=deposit_history_df['amount'],
        name='Deposit Amount'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Line(
        x=deposit_history_df['date'],
        y=deposit_history_df['cumsum_amount'],
        name='Total Amount'
    ),
    row=1, col=1
)

fig.add_vline(
    x=datetime.datetime.strptime("2022-09-15", "%Y-%m-%d").timestamp() * 1000,
    line_dash='dot',
    annotation_position="right",
    annotation_text="The Merge: Sep 15, 2022"
)

fig.update_layout(dict(template='simple_white',
                  title='Staked Ether Time Series', width=800, height=600))

fig.show()
